# Assignment #5 - Jackie Woodlief & Jeff Sharpe

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, Imputer
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

/home/shelob/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Big Picture

Based on the past performance of the 2015 flight dataset (from assignment 3), we saw that the non-linear SVM's performed well on the data set; however, that was due in part because we reduced the size of our data set by quite a bit. If the Gaussian RBF with grid search can yield such good results on a small - medium size and complex data set, we can imagine that since neural networks are better at dealing with large, complex data sets, then the performance may even improve. The main reason being is that there will be more data. With such a big, complex data set, a neural network seems like the most appropriate approach.

compare 2 different airlines, maybe

# Get the Data / Info

We are using the flight data set because it is our largest data set, it's massive, with almost 6,000,000 rows and 31 columns. In the past we couldn't use the entire data set because it was too big for our computers to handle efficiently. In addition, we already know that this is a complex data set due to our past assignments involving it and visualizing it.



https://www.kaggle.com/usdot/flight-delays/data

Click on the above link, and download flights.csv, as that is the data set I used for this assignment!

In [2]:
 #Let's read in our data

flights_data = pd.read_csv("flights.csv")

/home/shelob/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train_set_throwaway, flights = train_test_split(flights_data, test_size = 0.017, random_state = 42)
del train_set_throwaway # since we don't them, we can go ahead and delete them from the memory!

In [4]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98925 entries, 1508570 to 450748
Data columns (total 31 columns):
YEAR                   98925 non-null int64
MONTH                  98925 non-null int64
DAY                    98925 non-null int64
DAY_OF_WEEK            98925 non-null int64
AIRLINE                98925 non-null object
FLIGHT_NUMBER          98925 non-null int64
TAIL_NUMBER            98666 non-null object
ORIGIN_AIRPORT         98925 non-null object
DESTINATION_AIRPORT    98925 non-null object
SCHEDULED_DEPARTURE    98925 non-null int64
DEPARTURE_TIME         97460 non-null float64
DEPARTURE_DELAY        97460 non-null float64
TAXI_OUT               97403 non-null float64
WHEELS_OFF             97403 non-null float64
SCHEDULED_TIME         98925 non-null float64
ELAPSED_TIME           97137 non-null float64
AIR_TIME               97137 non-null float64
DISTANCE               98925 non-null int64
WHEELS_ON              97339 non-null float64
TAXI_IN                97339

In [5]:
flights.describe()

,YEAR,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,...,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
count,98925.0,98925.000000,98925.000000,98925.000000,98925.000000,98925.000000,97460.000000,97460.000000,97403.000000,97403.000000,...,98925.000000,97339.000000,97137.000000,98925.000000,98925.000000,18163.000000,18163.000000,18163.000000,18163.000000,18163.000000
mean,2015.0,6.516998,15.720718,3.907506,2179.155876,1328.242568,1333.406146,9.363411,16.043756,1355.475509,...,1491.153035,1475.228541,4.424061,0.002527,0.015547,13.541540,0.098552,18.807080,23.414524,2.811925
std,0.0,3.407106,8.782202,1.986356,1764.577362,483.378648,495.865855,36.999720,8.939410,497.354082,...,507.813903,525.762371,39.290706,0.050208,0.123716,27.541651,2.347444,49.214433,42.637061,20.486393
min,2015.0,1.000000,1.000000,1.000000,1.000000,3.000000,1.000000,-36.000000,1.000000,1.000000,...,1.000000,1.000000,-70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2015.0,4.000000,8.000000,2.000000,730.000000,919.000000,921.000000,-5.000000,11.000000,935.000000,...,1109.000000,1058.000000,-13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2015.0,7.000000,16.000000,4.000000,1687.000000,1325.000000,1329.000000,-2.000000,14.000000,1341.000000,...,1518.000000,1511.000000,-5.000000,0.000000,0.000000,2.000000,0.000000,2.000000,3.000000,0.000000
75%,2015.0,9.000000,23.000000,6.000000,3245.000000,1730.000000,1738.000000,7.000000,19.000000,1752.000000,...,1915.000000,1915.000000,8.000000,0.000000,0.000000,18.000000,0.000000,18.000000,29.000000,0.000000
max,2015.0,12.000000,31.000000,7.000000,7438.000000,2359.000000,2400.000000,1576.000000,225.000000,2400.000000,...,2359.000000,2400.000000,1557.000000,1.000000,1.000000,536.000000,142.000000,1557.000000,817.000000,865.000000


In [6]:
flights.head(5)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
1508570,2015,4,7,2,EV,4900,N759EV,FWA,DTW,1340,...,1423.0,-13.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
363270,2015,1,24,6,AS,611,N413AS,LAS,SEA,1910,...,2133.0,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3003945,2015,7,8,3,WN,1483,N463WN,OAK,SEA,630,...,812.0,-8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2291425,2015,5,26,2,WN,193,N7745A,STL,DAL,810,...,1222.0,152.0,0,0,NaN,0.0,0.0,0.0,152.0,0.0
2973084,2015,7,6,1,UA,253,N213UA,IAH,HNL,1000,...,1316.0,-2.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Let's make a copy just in case we need to refer back to the original

copy = pd.DataFrame.copy(flights)

In [8]:
# Based on the correlation matrix of airlines, used from a previous run, we determined that these features
# we not very correlated with determining the airline, so we decided to drop these features

copy = copy.drop("CANCELLATION_REASON", 1)
# copy = copy.drop("TAIL_NUMBER", 1)
copy = copy.drop("YEAR", 1)
copy = copy.drop("DAY", 1)
copy = copy.drop("MONTH", 1)
# copy = copy.drop("FLIGHT_NUMBER", 1)
copy = copy.drop("DAY_OF_WEEK", 1)
copy = copy.drop("DIVERTED", 1)
copy = copy.drop("CANCELLED", 1)
copy = copy.drop("AIR_SYSTEM_DELAY", 1)
copy = copy.drop("SECURITY_DELAY", 1)
copy = copy.drop("WHEELS_OFF", 1)
copy = copy.drop("SCHEDULED_DEPARTURE", 1)
copy = copy.drop("WEATHER_DELAY", 1)
copy = copy.drop("AIRLINE_DELAY", 1)
copy = copy.drop("WHEELS_ON", 1)
copy = copy.drop("SCHEDULED_ARRIVAL", 1)
copy = copy.drop("ARRIVAL_TIME", 1)
copy = copy.drop("DEPARTURE_TIME", 1)
copy = copy.drop("LATE_AIRCRAFT_DELAY", 1)
copy = copy.drop("ARRIVAL_DELAY", 1)
copy = copy.drop("DEPARTURE_DELAY", 1)

In [9]:
# copy = copy.reset_index(drop = True)

In [10]:
copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98925 entries, 1508570 to 450748
Data columns (total 11 columns):
AIRLINE                98925 non-null object
FLIGHT_NUMBER          98925 non-null int64
TAIL_NUMBER            98666 non-null object
ORIGIN_AIRPORT         98925 non-null object
DESTINATION_AIRPORT    98925 non-null object
TAXI_OUT               97403 non-null float64
SCHEDULED_TIME         98925 non-null float64
ELAPSED_TIME           97137 non-null float64
AIR_TIME               97137 non-null float64
DISTANCE               98925 non-null int64
TAXI_IN                97339 non-null float64
dtypes: float64(5), int64(2), object(4)
memory usage: 9.1+ MB


In [11]:
encoder = LabelEncoder() #AIRLINE, DESTINATION_AIRPORT, ORIGIN_AIRPORT

In [16]:
airline_cat = copy["AIRLINE"]
destination_cat = copy["DESTINATION_AIRPORT"]
origin_cat = copy["ORIGIN_AIRPORT"]
tail_cat = copy["TAIL_NUMBER"]
flight_cat = copy["FLIGHT_NUMBER"]

In [17]:
destination_cat = destination_cat.apply(str)
origin_cat = origin_cat.apply(str)
flight_cat = flight_cat.apply(str)
tail_cat = tail_cat.apply(str)

In [18]:
# destination_cat = destination_cat.fillna('Other Dest')
# origin_cat = origin_cat.fillna('Other Org')

In [19]:
airline_cat_encoded = encoder.fit_transform(airline_cat)
destination_cat_encoded = encoder.fit_transform(destination_cat)
origin_cat_encoded = encoder.fit_transform(origin_cat)
tail_cat_encoded = encoder.fit_transform(tail_cat)
flight_cat_encoded = encoder.fit_transform(flight_cat)

In [20]:
copy["AIRLINE"]= airline_cat_encoded
copy["DESTINATION_AIRPORT"] = destination_cat_encoded
copy["ORIGIN_AIRPORT"] = origin_cat_encoded
copy["FLIGHT_NUMBER"] = flight_cat_encoded
copy["TAIL_NUMBER"] = tail_cat_encoded

In [21]:
copy.head(3)

,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,TAXI_OUT,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,TAXI_IN
1508570,4,4229,3255,387,362,13.0,56.0,48.0,30.0,128,5.0
363270,1,5435,1551,440,545,14.0,155.0,155.0,131.0,867,10.0
3003945,13,537,1712,488,545,8.0,110.0,106.0,92.0,672,6.0


In [22]:
imputer = Imputer(strategy = "median")

In [24]:
imputer.fit(copy)
X = imputer.transform(copy)
transform_copy = pd.DataFrame(X, columns = copy.columns)
transform_copy

,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,TAXI_OUT,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,TAXI_IN
0,4.0,4229.0,3255.0,387.0,362.0,13.0,56.0,48.0,30.0,128.0,5.0
1,1.0,5435.0,1551.0,440.0,545.0,14.0,155.0,155.0,131.0,867.0,10.0
2,13.0,537.0,1712.0,488.0,545.0,8.0,110.0,106.0,92.0,672.0,6.0
3,13.0,1033.0,3384.0,560.0,350.0,6.0,100.0,99.0,85.0,546.0,8.0
4,10.0,1700.0,545.0,418.0,410.0,16.0,498.0,488.0,461.0,3904.0,11.0
5,3.0,424.0,4550.0,414.0,289.0,13.0,70.0,49.0,28.0,151.0,8.0
6,10.0,510.0,664.0,418.0,571.0,13.0,137.0,120.0,100.0,787.0,7.0
7,1.0,5893.0,2013.0,543.0,507.0,11.0,171.0,163.0,145.0,1107.0,7.0
8,0.0,0.0,3509.0,432.0,444.0,34.0,385.0,381.0,341.0,2475.0,6.0
9,5.0,5981.0,530.0,223.0,71.0,9.0,249.0,222.0,204.0,1607.0,9.0


In [ ]:
%matplotlib inline
transform_copy.hist(bins = 50, figsize = (20, 15))
plt.show()

In [25]:
correlation_matrix = transform_copy.corr()

In [26]:
correlation_matrix["AIRLINE"].sort_values(ascending = False)

AIRLINE                1.000000
FLIGHT_NUMBER          0.114277
TAIL_NUMBER            0.100310
ORIGIN_AIRPORT         0.080193
DESTINATION_AIRPORT    0.079925
TAXI_IN               -0.078662
DISTANCE              -0.084596
AIR_TIME              -0.088266
SCHEDULED_TIME        -0.107270
ELAPSED_TIME          -0.111724
TAXI_OUT              -0.165614
Name: AIRLINE, dtype: float64

In [27]:
train_set, test_set = train_test_split(transform_copy, test_size = 0.2, random_state = 42)

In [28]:
train_set.head(3)

,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,TAXI_OUT,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,TAXI_IN
87989,3.0,913.0,2553.0,362.0,507.0,15.0,253.0,225.0,206.0,1671.0,4.0
41349,4.0,4445.0,2536.0,333.0,484.0,18.0,125.0,124.0,101.0,622.0,5.0
26546,7.0,2393.0,2468.0,361.0,496.0,11.0,89.0,88.0,51.0,299.0,26.0


In [29]:
test_set.head(3)

,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,TAXI_OUT,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,TAXI_IN
75611,11.0,4184.0,311.0,505.0,496.0,15.0,202.0,194.0,173.0,1440.0,6.0
24040,3.0,1596.0,4308.0,543.0,442.0,19.0,142.0,135.0,108.0,867.0,8.0
24138,13.0,1519.0,3904.0,461.0,350.0,8.0,150.0,129.0,115.0,793.0,6.0


In [30]:
# Now we need to split up into test and train sets based on the airline (multi-class)

y_train = train_set["AIRLINE"]
y_test = test_set["AIRLINE"]
x_test = test_set.drop("AIRLINE", 1)
x_train = train_set.drop("AIRLINE", 1)


# We need to convert to float32 due to a warning we get when we do the dnn_clf
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train.astype('float32')

In [ ]:
y_train.head(2)

In [ ]:
x_train.head(2)

# Large-Scale Visualization

Below we will be visualizing different aspects of our data to demonstrate that it is in fact a complex data set!

In [31]:
# Before we do anything, in order to feed the various clf's our labels, we must first convert them from
# float to integers

y_train_int = y_train.astype(int)
y_test_int = y_test.astype(int)

In [ ]:
# x_train_ds = x_train.values
# x_test_ds = x_test.values

## Fully connected network with 2 hidden layers

This is the neural network that we start with!

Also, just to note, since we don't specify here, the activation function being used is the relu!

In [ ]:
x_train.shape

In [52]:
# Since there are 14 different airlines, we need to change n_classes to 14, default seems to be 10
# NOTE: This will take a few minutes!!

feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(x_train)
dnn_clf_hidden = tf.contrib.learn.DNNClassifier(hidden_units = [300, 150], n_classes = 14, feature_columns = feature_cols)
dnn_clf_hidden = tf.contrib.learn.SKCompat(dnn_clf_hidden)
dnn_clf_hidden.fit(x_train, y_train_int, batch_size = 50, steps = 15000) # 10,000 (46%)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8ecf17b2e8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmp3a9xktor'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp3a9xktor/model.ckpt.
INFO:tensorflow:loss = 414.89374, step = 1
INFO:tensorflow:global_step/sec: 89.4189
INFO:tensorflow:loss = 2.5871003, step = 101 (1.123 sec)
INFO:tensorflow:global_step/sec: 87.4342
INFO:tensorflow:loss

INFO:tensorflow:global_step/sec: 88.4173
INFO:tensorflow:loss = 1.5962082, step = 7301 (1.131 sec)
INFO:tensorflow:global_step/sec: 87.9755
INFO:tensorflow:loss = 1.6433401, step = 7401 (1.137 sec)
INFO:tensorflow:global_step/sec: 91.9078
INFO:tensorflow:loss = 1.6826012, step = 7501 (1.089 sec)
INFO:tensorflow:global_step/sec: 87.6513
INFO:tensorflow:loss = 1.7555861, step = 7601 (1.139 sec)
INFO:tensorflow:global_step/sec: 88.2755
INFO:tensorflow:loss = 1.8250775, step = 7701 (1.134 sec)
INFO:tensorflow:global_step/sec: 91.0319
INFO:tensorflow:loss = 1.5791428, step = 7801 (1.100 sec)
INFO:tensorflow:global_step/sec: 89.934
INFO:tensorflow:loss = 1.4345944, step = 7901 (1.112 sec)
INFO:tensorflow:global_step/sec: 90.1541
INFO:tensorflow:loss = 1.4475104, step = 8001 (1.108 sec)
INFO:tensorflow:global_step/sec: 89.0457
INFO:tensorflow:loss = 1.3728611, step = 8101 (1.123 sec)
INFO:tensorflow:global_step/sec: 91.3406
INFO:tensorflow:loss = 1.5740453, step = 8201 (1.097 sec)
INFO:tensor

SKCompat()

In [53]:
# With a smaller data set of about 100,000 entries, accuracy is about 41%
# it goes down when more entries are added it seems

y_predict_hidden = dnn_clf_hidden.predict(x_test)
accuracy_score(y_test_int, y_predict_hidden['classes'])

INFO:tensorflow:Restoring parameters from /tmp/tmp3a9xktor/model.ckpt-15000


0.47187263078089464

In [ ]:
y_predict_hidden_train = dnn_clf_hidden.predict(x_train)

# c = tf.confusion_matrix(y_train_int, y_predict_hidden_train['classes'])

with tf.Session():
    print('Confusion Matrix: \n\n', tf.Tensor.eval(tf.confusion_matrix(y_train_int, y_predict_hidden_train['classes']),feed_dict=None, session=None))

# Testing out some Activation Functions

Our fully connected network with 2 hidden layers didn't perform to well, so let's experiment with some activation functions!

The first alternate activation function that we will try is the sigmoid, after than we will try the tanh; in both cases the accuracy decreased from the default relu function.

In [ ]:
dnn_clf_sig = tf.contrib.learn.DNNClassifier(hidden_units = [300, 150], n_classes = 14, activation_fn="sigmoid", feature_columns = feature_cols)
dnn_clf_sig = tf.contrib.learn.SKCompat(dnn_clf_sig)
dnn_clf_sig.fit(x_train, y_train_int, batch_size = 50, steps = 10000) # 10,000 steps

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8ecd3f99e8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpp1pughzk'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpp1pughzk/model.ckpt.
INFO:tensorflow:loss = 3.0548224, step = 1
INFO:tensorflow:global_step/sec: 83.0795
INFO:tensorflow:loss = 2.076484, step = 101 (1.208 sec)
INFO:tensorflow:global_step/sec: 89.8761
INFO:tensorflow:loss 

In [ ]:
y_predict_sig = dnn_clf_sig.predict(x_test)
accuracy_score(y_test_int, y_predict_sig['classes'])

## Now let's try out tanh activation function

In [ ]:
dnn_clf_tanh = tf.contrib.learn.DNNClassifier(hidden_units = [300, 150], n_classes = 14, activation_fn="tanh", feature_columns = feature_cols)
dnn_clf_tanh = tf.contrib.learn.SKCompat(dnn_clf_tanh)
dnn_clf_tanh.fit(x_train, y_train_int, batch_size = 50, steps = 15000) # 10,000 steps

In [ ]:
y_predict_tanh = dnn_clf_tanh.predict(x_test)
accuracy_score(y_test_int, y_predict_tanh['classes'])

# Tweaking the Model Parameters to Improve Performance

In [ ]:
x_test.info()

# Execution Phase of DNN

In [ ]:
len(y_train) # This is the amount of times we will need to iterate through later

In [ ]:
len(train_set)

In [ ]:
# n_epochs = 40
# batch_size = 50
# init=tf.global_variables_initializer()

In [ ]:
# with tf.Session() as sess:
#     init.run()
#     for epoch in range(n_epochs):
#         for iteration in range(len(train_set) // batch_size):
#             x_batch = x_train.next_batch(batch_size)
#             y_batch = y_train_int.next_batch(batch_size)
#             sess.run(training_op, feed_dict = {X: x_batch, y: y_batch})
#         acc_train = accuracy.eval(feed_dict = {X: x_batch, y: y_batch})
#         acc_test = accuracy.eval(feed_dict = {X: x_test, y: y_test_int})
#         print(epoch, "Train Accuracy:", acc_train, "Test Accuracy:", acc_test)
    
#     save_path = saver.save(sess, "./my_model_final.ckpt")